In [1]:
import mibian
import datetime
import pandas as pd

def calc_implied_vol_delta_and_parity(row, underlying_price, days_to_expiry):
    interest_rate = 0.046
    delta_type = {'C': 'callDelta', 'P': 'putDelta', 'ATM Avg Put/Call': 'putDelta'}

    def calculate_mibian_bs(call_price, put_price, implied_vol=None):
        if implied_vol is None:
            ###print(f'Call Price: {call_price}, Put Price: {put_price}')
            c = mibian.BS([underlying_price, row['strike'], interest_rate, days_to_expiry], callPrice=call_price, putPrice=put_price)
            implied_vol = c.impliedVolatility
        c = mibian.BS([underlying_price, row['strike'], interest_rate, days_to_expiry], volatility=implied_vol)
        delta = getattr(c, delta_type[row['option_type']])
        return implied_vol, delta

    call_price, put_price = (row['mid_quote'], None) if row['option_type'] == 'C' else (None, row['mid_quote'])
    implied_vol, delta = calculate_mibian_bs(call_price, put_price)


    return pd.Series([implied_vol, delta])

In [2]:
import pandas as pd

def calculate_quantity_per_strike(row, vix_index):
    # Calculate the quantity per strike
    q  = (10000 *row.per_strike_constants)/(2*vix_index) 
    return q


expiry_date = '20230317'
observation_date = '2023-02-07'
inst_type = 'future'
working_dataset = f'./result-set/{observation_date}/{inst_type}/dataset_{expiry_date}.csv'
df = pd.read_csv(working_dataset)
vix_index = df['vix_forward'].iloc[0]
print("vix_index: ", vix_index)

df['quantity'] = df.apply(calculate_quantity_per_strike, axis=1, args=(vix_index,))
df.to_csv(working_dataset, index=False)

vix_index:  19.793164332117087


In [3]:
import pandas as pd
option_expiry = '20230317'
trade_date = '2023-02-07'
settlement_date = '2023-02-15'


# Load the datasets
df1 = pd.read_csv(f'./result-set/{trade_date}/future/dataset_{option_expiry}.csv')
df2 = pd.read_csv(f'./result-set/{settlement_date}/future/dataset_complete_{option_expiry}.csv')

def get_future_price_new(row, df2):
    if row['option_type'] == 'ATM Avg Put/Call':
        matching_rows = df2[df2['strike'] == row['strike']]
        if not matching_rows.empty:
            future_price = matching_rows['mid_quote'].mean()
            pl = row['quantity'] * (future_price - row['mid_quote'])
            return pd.Series([future_price, pl])
    else:
        matching_row = df2[(df2['strike'] == row['strike']) & (df2['option_type'] == row['option_type'])]
        if not matching_row.empty:
            future_price = matching_row['mid_quote'].values[0]
            pl = row['quantity'] * (future_price - row['mid_quote'])
            return pd.Series([future_price, pl])
    return pd.Series([None, None])

# Create the new column
df1[['future_prices', 'P&L']] = df1.apply(get_future_price_new, args=(df2,), axis=1)
df_final = df1[['strike', 'option_type', 'mid_quote', 'future_prices', 'quantity','vix_forward', 'P&L']].copy()
df_final.to_csv(f'./result-set/{trade_date}/P&L/dataset_{option_expiry}.csv', index=False)

In [ ]:
import pandas as pd
option_expiry_near = '20230421'
option_expiry_next = '20230428'
trade_date = '2023-03-21'

# Load the datasets
df_near = pd.read_csv(f'./result-set/{trade_date}/P&L/dataset_{option_expiry_near}.csv')
df_next = pd.read_csv(f'./result-set/{trade_date}/P&L/dataset_{option_expiry_next}.csv')

df = pd.concat([df_near, df_next])
df.to_csv(f'./result-set/{trade_date}/P&L/consolidated-dataset_{trade_date}.csv', index=False)


In [ ]:
import os
import pandas as pd

# Specify the directory you want to read from
directory = './result-set/P&L_across_days/'

# Create a list to hold all the dataframes
dfs = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):  # Assuming the files are CSVs
        # Read the file into a dataframe
        df = pd.read_csv(os.path.join(directory, filename))

        # Add the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes in the list
consolidated_df = pd.concat(dfs, ignore_index=True)

# Save the consolidated dataframe to a new CSV file
consolidated_df.to_csv('./result-set/P&L_across_days/consolidated.csv', index=False)

In [159]:
import os
import pandas as pd
import vix_utils
import glob
import pandas as pd

# Specify the directory you want to read from
parent_directory = './result-set/'
def calculate_greeks_add_future_prices():
    
    # Create a dictionary to hold the dataframes
    dataframes = {}

    # Iterate over all subdirectories in the parent directory
    for trade_date in os.listdir(parent_directory):
        # Construct the path to the 'future' directory
        future_directory = os.path.join(parent_directory, trade_date, 'future')

        # Check if the 'future' directory exists
        if os.path.isdir(future_directory):
            # Iterate over all files in the 'future' directory
            for filename in os.listdir(future_directory):
                # Check if the file matches the pattern 'dataset_*.csv' and does not start with 'dataset_complete_'
                if filename.startswith('dataset_') and not filename.startswith('dataset_complete_') and filename.endswith('.csv'):
                    # Construct the full file path
                    file_path = os.path.join(future_directory, filename)

                    # Read the file into a dataframe
                    df = pd.read_csv(file_path)
                    expiry_date = filename.split('_')[1].split('.')[0]
                    vix_index = df['vix_forward'].iloc[0]
                    df['quantity'] = df.apply(calculate_quantity_per_strike, axis=1, args=(vix_index,))

                    df = add_vol_and_delta(df,trade_date, expiry_date)

                    df.to_csv(file_path, index=False)
    ######################################################################################################
                    # Split trade_date into year, month, and day
                    year, month, day = trade_date.split('-')
                    future_settlement_date = vix_utils.vix_futures_expiry_date_from_trade_date(int(year), int(month), int(day), 1).strftime('%Y-%m-%d')
                    future_settlement_directory = os.path.join(parent_directory, future_settlement_date, 'future')

                    filtered_df = vix_futures[(vix_futures['Trade Date'] == trade_date) & (vix_futures['Tenor_Monthly'] == 1.0)]
                    settlement_df = vix_futures[(vix_futures['Trade Date'] == future_settlement_date) & (vix_futures['Tenor_Monthly'] == 1.0)]
                    if not filtered_df.empty:
                        vix_futures_on_trade_day = filtered_df['Close'].values[0]
                        vix_futures_on_settlement_day = settlement_df['Close'].values[0]
    ######################################################################################################                
                    
                    #complete_file_path = os.path.join(future_directory, 'dataset_complete_' + filename[8:])
                    complete_file_path = os.path.join(future_settlement_directory, 'dataset_complete_' + filename[8:])
                    if os.path.isfile(complete_file_path):
                        
                        df_complete = pd.read_csv(complete_file_path)
                        df[['future_prices', 'P&L']] = df.apply(get_future_price_new, args=(df_complete,), axis=1)
                        df_final = df[['strike', 'option_type','per_strike_constants', 'mid_quote', 'future_prices', 'quantity','vix_forward','Implied Vol','delta']].copy()
                        # Construct the path to the 'P&L' directory
                        pl_directory = future_directory.replace('future', 'P&L')

                        filtered_df = vix_futures[(vix_futures['Trade Date'] == trade_date) & (vix_futures['Tenor_Monthly'] == 1.0)]
                        if not filtered_df.empty:
                            vix_futures_on_trade_day = filtered_df['Close'].values[0]
                            df_final['vix_future_trade_day'] = vix_futures_on_trade_day
                            df_final['vix_future_settlment_day'] = vix_futures_on_settlement_day

                            df_final_abridged = df_final[['strike', 'option_type','per_strike_constants', 'mid_quote', 'future_prices', 
                                                          'quantity','vix_forward','vix_future_trade_day', 'vix_future_settlment_day','Implied Vol','delta']].copy()

                        # Check if the 'P&L' directory exists, if not, create it
                        if not os.path.exists(pl_directory):
                            os.makedirs(pl_directory)
                        # Save the merged dataframe in the 'P&L' directory
                        df_final.to_csv(os.path.join(pl_directory, filename), index=False)
                        ######df_final_abridged.to_csv(os.path.join(pl_directory, 'abridged_' + filename), index=False)

                    prefix = 'filtered_delta_strikes_'
                    new_file_path = file_path.replace('future', 'P&L')
                    df_filter = pd.read_csv(new_file_path)
                    ###df_filter = filter_delta_trial(df_filter)
                    df_filter = filter_delta_trial_latest(df_filter)
                    #df_filter = filter_delta_new(df_filter)
                    

                    df_filter = df_filter[['strike', 'option_type','per_strike_constants', 'mid_quote', 'future_prices','vix_forward',
                                          'vix_future_trade_day', 'vix_future_settlment_day','original_quantity', 'quantity_added','quantity',  'Implied Vol','delta']]
                    df_filter.to_csv(os.path.join(pl_directory, prefix + filename), index=False)


        pl_dir = future_directory.replace('future', 'P&L')
        print("pl_dir: ", pl_dir)
        file_patterns = ['dataset_*.csv', 'filtered_delta_strikes_dataset_*.csv']

        # Loop over the file patterns
        for file_pattern in file_patterns:
            # Construct the full file pattern
            full_file_pattern = os.path.join(pl_dir, file_pattern)

            # Get a list of all files that match the pattern
            file_list = glob.glob(full_file_pattern)

            # Check if the file list is empty
            if not file_list:
                print(f"No files found with the pattern {file_pattern}.")
            else:
                # Initialize an empty list to store the dataframes
                df_list = []

                # Loop over the file list
                for file in file_list:
                    # Load the file into a DataFrame and append it to the list
                    df_list.append(pd.read_csv(file))

                # Concatenate all the dataframes in the list
                df = pd.concat(df_list)
                #Add P&L column
                df['P&L-per-strike'] = df.apply(pl_per_strike, axis=1)


                # Create a unique output file name based on the file pattern
                output_file_name = file_pattern.replace('*', 'concatenated')

                # Save the concatenated dataframe to a new file
                df.to_csv(os.path.join(pl_dir, output_file_name), index=False)
                
                ####Total P&L contribution
                # future_pl, basket_pl, total_pl, strategy = pl_per_day(df)
                # #####################
                # df.to_csv(os.path.join(pl_dir, 'P&L_calculator.csv'), index=False)
                # ##########################    
                # parent_directory_temp = os.path.dirname(pl_directory)
                # trade_date = parent_directory_temp.split('/')[-1]
                # df_per_day_PL = pd.DataFrame({
                #     'trade_date': [trade_date],
                #     'vix-forward': [df['vix_forward'].iloc[0]],  
                #     'vix-futures-trade-day': [df['vix_future_trade_day'].iloc[0]],
                #     'vix-futures-settlement-day': [df['vix_future_settlment_day'].iloc[0]],
                #     'future-P&L':[future_pl], 
                #     'basket-P&L' : [basket_pl],
                #     'total-P&L' : [total_pl],
                #     'strategy' : [strategy]
                #     })
                # file_name = f'P&L_for_{trade_date}.csv'
                # directory = os.path.join('./result-set', 'P&L-across-days')
                # # Create the directory if it does not exist
                # os.makedirs(directory, exist_ok=True)
                # file_path = os.path.join(directory, file_name)
                # df_per_day_PL.to_csv(file_path, index=False)



In [5]:
def add_vol_and_delta(df, trade_date, expiry_date):
    days_to_expiry = (datetime.datetime.strptime(expiry_date, "%Y%m%d") - datetime.datetime.strptime(trade_date, "%Y-%m-%d")).days
    forward = df[df['option_type'] == 'ATM Avg Put/Call']['strike'].values[0]
    df[['Implied Vol', 'delta']] = df.apply(calc_implied_vol_delta_and_parity, args=(forward, days_to_expiry), axis=1)

    return df

In [152]:
def pl_per_day(df):

    vix_forward = df['vix_forward'].iloc[0]
    vix_future_trade_day = df['vix_future_trade_day'].iloc[0]
    vix_future_settlment_day = df['vix_future_settlment_day'].iloc[0]
    
    def calculate(row):
        return (row['future_prices'] - row['mid_quote']) * row['quantity']
    
    basket_pl = df.apply(calculate, axis=1).sum()
    future_pl = (vix_future_settlment_day - vix_future_trade_day)
    
    if vix_forward > vix_future_trade_day:
        total_pl = future_pl - basket_pl
        basket_pl = -basket_pl
        future_pl = future_pl
        comment = 'Buy The future and Sell the basket'
    else:
        total_pl = basket_pl - future_pl
        basket_pl = basket_pl
        future_pl = -future_pl
        comment = 'Buy the basket and Sell the future'

    return future_pl, basket_pl, total_pl, comment

In [120]:
def pl_per_strike(row):

    if row['vix_forward'] > row['vix_future_trade_day']:
        pl = (row['future_prices'] - row['mid_quote'])* row['quantity']
        ##comment = 'Buy future price sell the basket'
    else:
        pl =  (row['mid_quote'] - row['future_prices']) * row['quantity']
        ##comment = 'Buy basket sell future'
    return pl  
           

In [114]:
def redistribute_quantities(df, selected_strikes):
    total_quantity_to_redistribute = df[~df.index.isin(selected_strikes.index)]['quantity'].sum()
    total_quantity_selected = selected_strikes['quantity'].sum()
    proportions = selected_strikes['quantity'] / total_quantity_selected

    print(proportions)

    selected_strikes['redistributed_quantity'] = proportions * total_quantity_to_redistribute
    return selected_strikes

In [102]:
from IPython.display import display, HTML

def filter_delta_trial(df, bounds = 0.25):
    
    
    # Create an empty DataFrame to store the selected rows
    selected_df = pd.DataFrame()
    df_calls = df[df['option_type'] == 'C']
    df_puts = df[df['option_type'] == 'P']

    def select_strikes(df, bounds = 0.1):

        # List of absolute delta values to filter by
        delta_values = [0.1, 0.25, 0.5]
        selected_df_inner = pd.DataFrame()

        # Loop over the delta values
        for delta in delta_values:
            # Calculate the upper and lower bounds for the delta value
            lower_bound = delta - delta * bounds
            upper_bound = delta + delta * bounds
            
            # Filter the dataframe for rows where the absolute value of 'delta' is within the bounds
            # and the 'strike' value ends with at least one zero
            delta_df = df[(df['delta'].abs() >= lower_bound) & (df['delta'].abs() <= upper_bound) & (df['strike'] % 10 == 0)]

            # If no rows found, select the row with the delta value closest to the desired delta
            if len(delta_df) == 0:
                delta_df = df[df['delta'].abs() == df['delta'].abs().sub(delta).abs().min()]

            # If there are multiple rows that satisfy the conditions, select the one with the highest quantity (most liquid)
            if len(delta_df) > 1:
                delta_df = delta_df[delta_df['quantity'] == delta_df['quantity'].max()]

            # Append the selected row to the selected dataframe
            selected_df_inner = pd.concat([selected_df_inner, delta_df])
            
        return selected_df_inner

    selected_df_calls = select_strikes(df_calls)
    selected_df_puts = select_strikes(df_puts)  
    selected_df = pd.concat([selected_df_calls, selected_df_puts])
      
    # Add a new column to store the original quantities
    selected_df['original_quantity'] = selected_df['quantity']

    # Calculate the total quantity from the original dataframe
    total_quantity = df['quantity'].sum()

    # Calculate the quantity to be removed
    removed_quantity = total_quantity - selected_df['quantity'].sum()

    # Calculate the quantity to be added to each row in the selected DataFrame
    selected_df['quantity_added'] = removed_quantity * (selected_df['quantity'] / selected_df['quantity'].sum())

    # Add the calculated quantity to the original quantity
    selected_df['quantity'] += selected_df['quantity_added']

    return selected_df

In [14]:
from IPython.display import display, HTML

def filter_delta_new(df, bounds = 0.1):
    display(HTML(df.to_html(index=False, border=0)))
    # List of absolute delta values to filter by
    delta_values = [0.1, 0.25, 0.5]

    # Create an empty DataFrame to store the selected rows
    selected_df = pd.DataFrame()
    calls_df = df[df['option_type'] == 'call']
    puts_df = df[df['option_type'] == 'put']

    # Loop over the delta values
    for delta in delta_values:
        # Calculate the upper and lower bounds for the delta value
        lower_bound = delta - delta * bounds
        upper_bound = delta + delta * bounds

        # Filter the dataframe for rows where the absolute value of 'delta' is within the bounds
        # and the 'strike' value ends with at least one zero
        delta_df = df[(df['delta'].abs() >= lower_bound) & (df['delta'].abs() <= upper_bound) & (df['strike'] % 10 == 0)]

        # If no rows found, select the row with the delta value closest to the desired delta
        if len(delta_df) == 0:
            delta_df = df[df['delta'].abs() == df['delta'].abs().sub(delta).abs().min()]

        # If there are multiple rows that satisfy the conditions, select the one with the highest quantity (most liquid)
        if len(delta_df) > 1:
            delta_df = delta_df[delta_df['quantity'] == delta_df['quantity'].max()]

        # Append the selected row to the selected dataframe
        selected_df = pd.concat([selected_df, delta_df])

    # Add a new column to store the original quantities
    selected_df['original_quantity'] = selected_df['quantity']

    # Calculate the total quantity from the original dataframe
    total_quantity = df['quantity'].sum()

    # Calculate the quantity to be removed
    removed_quantity = total_quantity - selected_df['quantity'].sum()

    # Calculate the quantity to be added to each row in the selected DataFrame
    selected_df['quantity_added'] = removed_quantity * (selected_df['quantity'] / selected_df['quantity'].sum())

    # Add the calculated quantity to the original quantity
    selected_df['quantity'] += selected_df['quantity_added']

    return selected_df

In [8]:
import os
import pandas as pd
import vix_utils, logging, asyncio

# Specify the directory you want to read from
vix_futures,vix_cash=await asyncio.gather(vix_utils.async_load_vix_term_structure(),vix_utils.async_get_vix_index_histories())

def calculate_pl(for_pl_directory, consolidated_pl_directory, filename_prefix):
    
    parent_directory = './result-set/'
    # Create a dictionary to hold the dataframes
    dataframes = {}

    # Iterate over all subdirectories in the parent directory
    for trade_date in os.listdir(parent_directory):
        # Construct the path to the 'P&L' directory
        pl_directory = os.path.join(parent_directory, trade_date, for_pl_directory)
        
        # Check if the 'P&L' directory exists
        if os.path.isdir(pl_directory):
            # Get the list of files in the 'P&L' directory
            ###files = [f for f in os.listdir(pl_directory) if os.path.isfile(os.path.join(pl_directory, f))]
            files = [f for f in os.listdir(pl_directory) if os.path.isfile(os.path.join(pl_directory, f)) and f.startswith(filename_prefix)]

            # Sort the files
            files.sort()

            # Check if there are at least two files
            if len(files) >= 2:
            # Construct the full file paths
                near_file_path = os.path.join(pl_directory, files[0])
                next_file_path = os.path.join(pl_directory, files[1])
                # Read the files into dataframes
                df_near = pd.read_csv(near_file_path)
                df_next = pd.read_csv(next_file_path)

                vix_forward = df_near['vix_forward'].iloc[0]
                sum_value_near = df_near['P&L'].sum()
                sum_value_next = df_next['P&L'].sum()
                sum_total = sum_value_near + sum_value_next
                filtered_df = vix_futures[(vix_futures['Trade Date'] == trade_date) & (vix_futures['Tenor_Monthly'] == 1.0)]
                if not filtered_df.empty:
                    vix_futures_on_trade_day = filtered_df['Close'].values[0]
                
                    year, month, day = trade_date.split('-')
                    settlement_date = vix_utils.vix_futures_expiry_date_from_trade_date(int(year), int(month), int(day), 1).strftime('%Y-%m-%d')
                    vix_futures_on_settlement_day = vix_futures[(vix_futures['Trade Date'] == settlement_date) & (vix_futures['Tenor_Monthly'] == 1.0)]['Close'].values[0]
                    vix_future_delta = vix_futures_on_settlement_day - vix_futures_on_trade_day
                    basket_pl = sum_total if vix_forward < vix_futures_on_trade_day else -sum_total
                    future_pl = vix_future_delta if vix_futures_on_trade_day < vix_forward  else -vix_future_delta

                    # Create a DataFrame
                    df_summary = pd.DataFrame({
                        'trade_date': [trade_date],
                        'Basket-Delta-near': [sum_value_near],
                        'Basket-Delta-next': [sum_value_next],
                        'Basket-Delta-total': [sum_total],
                        'vix-forward': [vix_forward],
                        'vix-futures-on-trade_day': [vix_futures_on_trade_day],
                        'vix-futures-on-settlement_day': [vix_futures_on_settlement_day],
                        'Future-Delta' : [vix_future_delta],
                        'Basket-P&L': basket_pl,
                        'Future-P&L': future_pl,
                        'Total-P&L': basket_pl + future_pl     
                    })

                    # Print the DataFrame
                    print(df_summary)
                    ###df_summary.to_csv(os.path.join(pl_directory, f'option_basket_P&L_{trade_date}.csv'), index=False)
                    df_summary.to_csv(os.path.join(consolidated_pl_directory, f'option_basket_P&L_{trade_date}.csv'), index=False)


Duplicates detected
                     Trade Date  Weekly  Tenor_Monthly  Tenor_Days  \
37833 2020-02-27 00:00:00-05:00    True            2.0          26   
37834 2020-02-27 00:00:00-05:00    True            2.0          26   
37848 2020-02-28 00:00:00-05:00    True            2.0          25   
37849 2020-02-28 00:00:00-05:00    True            2.0          25   

       Tenor_Trade_Days                    Expiry   Open   High    Low  Close  \
37833              19.0 2020-03-25 00:00:00-04:00  24.15  25.35  22.15  24.25   
37834              19.0 2020-03-25 00:00:00-04:00  24.15  25.35  22.15  24.25   
37848              18.0 2020-03-25 00:00:00-04:00  26.50  27.50  25.65  26.50   
37849              18.0 2020-03-25 00:00:00-04:00  26.50  27.50  25.65  26.50   

       Settle  Change  Total Volume  EFP  Open Interest  Year  MonthOfYear  \
37833  25.025   3.000            48    0             63  2020            3   
37834  25.025   3.000            48    0             63  2020      

In [162]:
def chain_of_responsibility(delta_filter):
    calculate_greeks_add_future_prices()
    pl_directory = 'P&L'
    consolidated_pl_directory = './result-set/P&L-across-days/'
    #calculate_pl(pl_directory, consolidated_pl_directory, filename_prefix='dataset_')
    #calculate_pl(pl_directory, consolidated_pl_directory, filename_prefix='filtered_delta_strikes_')

delta_filter = 0.24
chain_of_responsibility(delta_filter)

pl_dir:  ./result-set/2023-02-07\P&L
pl_dir:  ./result-set/2023-02-15\P&L
pl_dir:  ./result-set/P&L-across-days\P&L
No files found with the pattern dataset_*.csv.
No files found with the pattern filtered_delta_strikes_dataset_*.csv.


In [ ]:

# Specify the directory you want to read from
directory = './result-set/P&L-across-days/'

# Create a list to hold all the dataframes
dfs = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):  # Assuming the files are CSVs
        # Read the file into a dataframe
        df = pd.read_csv(os.path.join(directory, filename))

        # Add the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes in the list
consolidated_df = pd.concat(dfs, ignore_index=True)

# Save the consolidated dataframe to a new CSV file
consolidated_df.to_csv(f'{directory}/consolidated.csv', index=False)

import pandas as pd

# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(f'{directory}/pandas_conditional.xlsx', engine='xlsxwriter') as writer:
    # Convert the dataframe to an XlsxWriter Excel object.
    consolidated_df.to_excel(writer, sheet_name='Sheet1', index=False)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Define a format for the red color. 
    red_format = workbook.add_format({'bg_color': '#FF0000'})

    # Apply a conditional format to the cells in the 'Total-P&L' column.
    worksheet.conditional_format('A1:Z1000', {'type': 'formula',
                                              'criteria': '=$K1<0',
                                              'format': red_format})
    # Count the number of positive and negative 'Total-P&L' values.
    positive_count = (consolidated_df['Total-P&L'] > 0).sum()
    negative_count = (consolidated_df['Total-P&L'] < 0).sum()

    # Create a new DataFrame to hold the counts.
    counts_df = pd.DataFrame({
        'Condition': ['Total-P&L > 0', 'Total-P&L < 0'],
        'Count': [positive_count, negative_count]
    })

    # Write the counts DataFrame to a new sheet in the Excel file.
    counts_df.to_excel(writer, sheet_name='Counts', index=False)


print("Done")

In [ ]:
df_near =pd.read_csv('./result-set/2023-02-07/P&L/dataset_20230317.csv')
df_next =pd.read_csv('./result-set/2023-02-07/P&L/dataset_20230324.csv')
df_25_delta_near = pd.read_csv('./result-set/2023-02-07/P&L/filterd_delta_strikes_dataset_20230317.csv')
df_25_delta_next = pd.read_csv('./result-set/2023-02-07/P&L/filterd_delta_strikes_dataset_20230324.csv')


df_combined_near = df_25_delta_near.merge(df_near[['strike', 'quantity','future_prices']], on='strike', how='left')
df_combined_near.rename(columns={'quantity_y': 'old_quantity', 'quantity_x': 'quantity'}, inplace=True)
df_combined_near = df_combined_near[['strike', 'option_type', 'per_strike_constants', 'mid_quote', 'future_prices','old_quantity','quantity','vix_forward', 'Implied Vol','delta']].copy()
df_combined_near['Term'] = 'Near'

df_combined_next = df_25_delta_next.merge(df_next[['strike', 'quantity', 'future_prices']], on='strike', how='left')
df_combined_next.rename(columns={'quantity_y': 'old_quantity', 'quantity_x': 'quantity'}, inplace=True)
df_combined_next = df_combined_next[['strike', 'option_type', 'per_strike_constants','mid_quote', 'future_prices', 'old_quantity','quantity','vix_forward', 'Implied Vol','delta']].copy()
df_combined_next['Term'] = 'Next'

df_combined_final = pd.concat([df_combined_near, df_combined_next])

df_combined_near.to_csv('./result-set/2023-02-07/P&L/Final/filterd_delta_strikes_dataset_20230317.csv', index=False)
df_combined_next.to_csv('./result-set/2023-02-07/P&L/Final/filterd_delta_strikes_dataset_20230324.csv', index=False)
df_combined_final.to_csv('./result-set/2023-02-07/P&L/Final/filterd_delta_strikes_dataset.csv', index=False)



In [161]:
from IPython.display import display, HTML

def filter_delta_trial_latest(df, bounds = 0.25):
    # Create an empty DataFrame to store the selected rows
    selected_df = pd.DataFrame()
    df_calls = df[df['option_type'] == 'C']
    df_puts = df[df['option_type'] == 'P']

    delta_series = pd.Series(delta)               

    def select_strike(df, delta, bound):
    # Calculate the lower and upper bounds
    lower_bound = delta - bound
    upper_bound = delta + bound

    # Select rows where the absolute delta is within the bounds
    delta_df = df[(df['delta'].abs() >= lower_bound) & (df['delta'].abs() <= upper_bound)]

    # If no rows found within bounds, select the row with the delta value closest to the desired delta
    if len(delta_df) == 0:
        delta_df = df[df['delta'].abs() == df['delta'].abs().sub(delta).abs().min()]

    # Try to select the row with the strike that ends with a zero
    delta_df_zero_strike = delta_df[delta_df['strike'] % 10 == 0]

    # If there are multiple rows that satisfy the conditions, select the one with the highest quantity (most liquid)
    if len(delta_df_zero_strike) > 1:
        delta_df_zero_strike = delta_df_zero_strike[delta_df_zero_strike['quantity'] == delta_df_zero_strike['quantity'].max()]
    elif len(delta_df_zero_strike) == 0 and len(delta_df) > 1:
        delta_df = delta_df[delta_df['quantity'] == delta_df['quantity'].max()]

    # Return the selected row
    return delta_df_zero_strike if len(delta_df_zero_strike) > 0 else delta_df

    selected_strikes_calls = delta_series.apply(lambda d: select_strike(df_calls, d, bound))
    selected_strikes_puts = delta_series.apply(lambda d: select_strike(df_puts, d, bound))
    
    selected_df_calls = pd.concat(selected_strikes_calls.tolist())
    selected_df_puts = pd.concat(selected_strikes_puts.tolist())

    selected_df = pd.concat([selected_df_calls, selected_df_puts])
      
    # Add a new column to store the original quantities
    selected_df['original_quantity'] = selected_df['quantity']

    # Calculate the total quantity from the original dataframe
    total_quantity = df['quantity'].sum()

    # Calculate the quantity to be removed
    removed_quantity = total_quantity - selected_df['quantity'].sum()

    # Calculate the quantity to be added to each row in the selected DataFrame
    selected_df['quantity_added'] = removed_quantity * (selected_df['quantity'] / selected_df['quantity'].sum())

    # Add the calculated quantity to the original quantity
    selected_df['quantity'] += selected_df['quantity_added']

    return selected_df